In [0]:
!python -m spacy download el_core_news_sm

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd
%matplotlib inline
import numpy as np
import glob
from sklearn.feature_extraction.text import CountVectorizer
from datetime import datetime

# Διαβάζουμε το EmoLex



In [0]:
filepath = "https://raw.githubusercontent.com/datajour-gr/Data_journalism/master/week10/NRC_GREEK_Translated_6_2020.csv"
emolex_df = pd.read_csv(filepath)
emolex_df.head()

In [0]:
emolex_df.tail()

In [0]:
emolex_df = emolex_df.drop_duplicates(subset=['word'])
emolex_df = emolex_df.dropna()
emolex_df.reset_index(inplace = True)

In [0]:
emolex_df.tail()

In [0]:
import spacy
nlp = spacy.load('el_core_news_sm')

## Διαβάζουμε τα tweets



In [0]:
full_tweets_df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/TsiprasTweets080419.tsv", sep='\t')

In [0]:
len(full_tweets_df)

In [0]:
short_tweets = full_tweets_df[['created_at','full_text' ]]
short_tweets = short_tweets.rename(columns={'created_at': 'date', 'full_text': 'text'} )

In [0]:
short_tweets.tail()

In [0]:
short_tweets.dtypes

In [0]:
short_tweets['date'] = pd.to_datetime(short_tweets['date'], format='%a %b %d %H:%M:%S +0000 %Y')


In [0]:
short_tweets.dtypes

In [0]:
emolex_df['word'].head(3)

In [0]:
short_tweets['text'] = short_tweets['text'].str.replace(r'https?:\/\/.*[\r\n]*'," ")

In [0]:
short_tweets.head()

In [0]:
short_tweets.head()

In [0]:
vec = CountVectorizer(analyzer = 'word', vocabulary = emolex_df.word, 
                      strip_accents = 'unicode',  
                      stop_words= nlp.Defaults.stop_words,
                      ngram_range=(1, 2))

In [0]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#CountVectorizer(stop_words='english', binary=True)
# Θέλω να ελέγξω μόνο τις λέξεις που υπάρχουν στο συναισθηματικό λεξικό

#CountVectorizer(analyzer = 'word', 
#                strip_accents = 'unicode', 
#                token_pattern='[Α-Ωα-ωΆ-Ώά-ώ]{2,}', 
#                stop_words= lc_stop_words_df['stop_w'].tolist())


matrix = vec.fit_transform(short_tweets['text'])
vocab = vec.get_feature_names()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)
wordcount_df.head()

In [0]:
wordcount_df.shape

In [0]:
emolex_df.head()

In [0]:
emolex_df[emolex_df.Anger == 1].tail()

In [0]:
# Φτιάξε μια λίστα με positive words

angry_words = emolex_df[emolex_df.Anger == 1]['word']

positive_words = emolex_df[emolex_df.Positive == 1]['word']


# Φτιάξε μια λίστα με sadness words
sadness_words = emolex_df[emolex_df.Sadness == 1]['word']


# Φτιάξε μια λίστα με surprise words
surprise_words = emolex_df[emolex_df.Surprise == 1]['word']


# Φτιάξε μια λίστα με disgust words
disgust_words = emolex_df[emolex_df.Disgust == 1]['word']


# Φτιάξε μια λίστα με anticipation words
anticipation_words = emolex_df[emolex_df.Anticipation == 1]['word']


# Φτιάξε μια λίστα με negative words
negative_words = emolex_df[emolex_df.Negative == 1]['word']



# Φτιάξε μια λίστα με joy words
joy_words = emolex_df[emolex_df.Joy == 1]['word']


# Φτιάξε μια λίστα με trust words
trust_words = emolex_df[emolex_df.Trust == 1]['word']



# Φτιάξε μια λίστα με fear words
fear_words = emolex_df[emolex_df.Fear == 1]['word']




In [0]:
short_tweets.head(3)

In [0]:
short_tweets['anger'] = wordcount_df[angry_words].sum(axis=1)

short_tweets['positivity'] = wordcount_df[positive_words].sum(axis=1)


short_tweets['joy'] = wordcount_df[joy_words].sum(axis=1)


short_tweets['disgust'] = wordcount_df[disgust_words].sum(axis=1)



short_tweets['surprise'] = wordcount_df[surprise_words].sum(axis=1)

short_tweets['trust'] = wordcount_df[trust_words].sum(axis=1)


short_tweets['anticipation'] = wordcount_df[anticipation_words].sum(axis=1)


short_tweets['sadness'] = wordcount_df[sadness_words].sum(axis=1)

short_tweets['negative'] = wordcount_df[negative_words].sum(axis=1)

short_tweets['fear'] = wordcount_df[fear_words].sum(axis=1)




In [0]:
short_tweets

In [0]:
short_tweets.set_index('date' , inplace=True)

In [0]:
short_tweets['text'].str.len().groupby(by=short_tweets.index.month).mean().plot() 

In [0]:
short_tweets['text'].str.len().resample('A').mean().plot() 

In [0]:
short_tweets['positivity'].resample('A').mean().plot() 

In [0]:
short_tweets['negative'].resample('A').sum().plot(color = 'b',label = 'neg').legend() 

In [0]:
short_tweets['anger'].resample('M').sum().plot(figsize=(16,4))

In [0]:
ax = short_tweets['anger'].resample('M').sum().plot(figsize=(16,4), color = 'red')
short_tweets['joy'].resample('M').sum().plot(figsize=(16,4), ax = ax, color = 'green')

In [0]:
ax = short_tweets['positivity'].resample('M').sum().plot(figsize=(16,4), color = 'red')
short_tweets['negative'].resample('M').sum().plot(figsize=(16,4), ax = ax, color = 'green')
short_tweets['trust'].resample('M').sum().plot(figsize=(16,4), ax = ax, color = 'blue')

In [0]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [0]:
ax = short_tweets['positivity'].resample('M').mean().plot(figsize=(16,4), 
                                                          color = 'g', label='pos')
short_tweets['negative'].resample('M').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg', color = 'r')
short_tweets['trust'].resample('M').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust',color = 'blue').legend()